In [ ]:
from rtlsdr import RtlSdr
import pylab
import os
import sys
sys.path.insert(0, os.path.abspath('..'))


from util.plotting import compute_fft_plot, compute_fft_plot_from_sample_rate, compute_frequency_response
from util.filtering import compute_lpf_coeff, filter_complex_signal, filter_real_signal
from util.data_io import read_sdriq_data
from util.demodulation import demodulate_signal, chunked_demodulate_signal
import numpy as np
from scipy.io import wavfile

In [ ]:
# Crop and save the data. It's too much!
fs = 1920000
full_raw_file = "/Users/benjaminpattison/Documents/Projects/satNav/SDR_satellite_tracking/data/gqrx_20211121_204702_435103000_1920000_fc.raw"
cropped_raw_file = "/Users/benjaminpattison/Documents/Projects/satNav/SDR_satellite_tracking/data/gqrx_20211121_204702_435103000_1920000_fc_cropped.raw"

raw_signal = np.fromfile(full_raw_file, dtype="uint8")

total_cropped_length = 120  # [s]
cropped_start = 120  # [s]
cropped_start_index = 120 * fs
# We do 2x the expected length because the I and Q data are interleaved.
cropped_end_index = cropped_start_index + 2*total_cropped_length * fs
cropped_raw_signal = raw_signal[cropped_start_index:cropped_end_index]
raw_signal = None  # hack to clear the memory usage

np.save(cropped_raw_file, cropped_raw_signal)

# Arg! Numpy adds a non-optional .npy extension, but we want this to be consistent
# with our other names. Rename to be consistent...
os.rename(cropped_raw_file + ".npy", cropped_raw_file)

In [ ]:
fs = 1920000
center_frequency = 435103000

signal = read_sdriq_data(cropped_raw_file)
print(f"signal is of length {len(signal) / fs} seconds")

In [ ]:
signal_trimmed = signal[169:int(fs/9600)]

print("max:" , np.max(np.abs(signal_trimmed)))
print("min:" , np.min(np.abs(signal_trimmed)))
print("mean:" , np.mean(np.abs(signal_trimmed)))
print("max - min:" , np.max(np.abs(signal_trimmed)) - np.min(np.abs(signal_trimmed)))
print("max - mean:" , np.max(np.abs(signal_trimmed)) - np.mean(np.abs(signal_trimmed)))

pylab.figure(2,figsize=[10,10])
pylab.plot(np.abs(signal_trimmed) - np.mean(np.abs(signal_trimmed)))
pylab.title("Signal minus its mean")
pylab.show(2)

phase_angles = np.angle(signal_trimmed)
phase_angle_diff = np.diff(phase_angles)

pylab.figure(3,figsize=[10,10])
pylab.plot(phase_angles)
pylab.title("Phase angles")
pylab.show(3)

pylab.figure(4,figsize=[10,10])
pylab.plot(phase_angle_diff)
pylab.title("Diff of the phase angles")
pylab.show(4)

In [ ]:
# Low pass and sub-sample the signal to get it into a more reasonable sampling rate
filtered_signal = filter_complex_signal(
    signal=signal,
    cutoff_frequency=48E3,
    sample_rate=fs,
    apply_mean_shift=False
)
downsample_factor = 15
filtered_signal_downsampled = filtered_signal[::15]
downsampled_fs = int(fs / 15)

In [ ]:
# Now try out the filtering that we found in GNU radio code.
def conjugate_filter(input):
    # Product of the complex conjugate of the original signal and
    # the 1-sample-time-delayed signal.
    # Note, this will be 1 sample shorter than the input.
    return np.conjugate(input[1:]) * input[:-1]


In [ ]:
conjugate_down_sampled_filtered_signal = conjugate_filter(filtered_signal_downsampled)

In [ ]:
pylab.figure(1,figsize=[10,10])
pylab.specgram(filtered_signal_downsampled,Fc=0,Fs=downsampled_fs,NFFT=1024)
pylab.show()

In [ ]:
pylab.figure(1,figsize=[10,10])
pylab.specgram(conjugate_down_sampled_filtered_signal,Fc=0,Fs=downsampled_fs,NFFT=1024)
pylab.show()

In [ ]:
# Plot 1ms worth of samples of the original

pylab.figure()
pylab.plot(filtered_signal_downsampled[:int(downsampled_fs*0.001)])
pylab.show()

In [ ]:
# Plot 1ms worth of samples of the original

pylab.figure()
pylab.plot(conjugate_down_sampled_filtered_signal[:int(downsampled_fs*0.001)])
pylab.show()

In [ ]:
phase_angles = np.angle(conjugate_down_sampled_filtered_signal)
phase_angle_diff = np.unwrap(np.diff(phase_angles))

In [ ]:
# Plot 1ms worth of samples of the original

pylab.figure()
pylab.plot(phase_angles[:int(downsampled_fs*0.001)])
pylab.show()

In [ ]:
# Plot 1ms worth of samples of the original

pylab.figure()
pylab.plot(phase_angle_diff[:int(downsampled_fs*0.02)])
pylab.show()

In [ ]:
np.angle(conjugate_down_sampled_filtered_signal[:10])

In [ ]:


phase_angles = np.angle(filtered_signal_downsampled)
phase_angle_diff = np.unwrap(np.diff(phase_angles))
phase_angle_diff[:10]

In [ ]:
signal_cut = signal[30*fs:90*fs]
filtered_signal_cut = filter_complex_signal(signal=signal_cut, cutoff_frequency=2*9600, sample_rate=fs)
filtered_signal_down = filtered_signal_cut[0::int(fs/4/9600)]
fs_down = int(filtered_signal_down.shape[0]/60)
print(fs_down)
print(filtered_signal_cut.shape)
print(filtered_signal_down.shape)

In [ ]:
pylab.figure(1,figsize=[10,10])
pylab.specgram(signal_cut,Fc=center_frequency,Fs=fs,NFFT=16384,window=np.hamming(16384))
pylab.ylim((center_frequency-10e3,center_frequency+10e3))
pylab.show()

In [ ]:
angles = np.angle(filtered_signal_down)
# Difference the phase angle and convert to +/-pi range.
angle_diff = np.unwrap(np.diff(angles))
pylab.figure()
pylab.plot(angle_diff[0:50])
pylab.show()
"""filtered_angle_diff = filter_real_signal(signal=angle_diff, cutoff_frequency=20e3, sample_rate=fs)
angle_diff_downsampled = filtered_angle_diff[::int(fs/1200)]
rounded_samples = np.round(angle_diff_downsampled).astype(int)
print(np.max(rounded_samples))
print(np.min(rounded_samples))"""




In [ ]:
"""
window_size = 1 # second
for idx in [0]: #range(signal.shape[0]/(window_size*fs)):
    start_index = 0 + (window_size*fs*idx)
    stop_index = (window_size*fs*(idx+1))
    this_chunk = filtered_signal[start_index:stop_index]
    spectrum, frequencies, _, _ = pylab.specgram(this_chunk,NFFT=1024,Fc=center_frequency,Fs=fs)

import struct
format_string = ">b"
test_bytes = [struct.pack(format_string, i) for i in np.round(angle_diff_downsampled).astype(int)]
print(test_bytes[:100])
pylab.figure()
pylab.plot(np.round(angle_diff_downsampled).astype(int))
pylab.show()
"""
#print(np.round(angle_diff_downsampled[:100]).astype(int))
#print(angle_diff_downsampled[:100])